# Chatbot with LangChain Memory (Gemini Model)

This open-source notebook demonstrates how to build a conversational AI assistant step by step using LangChain and Google Generative AI (Gemini). It covers:

- Installing dependencies
- Basic model invocation
- Output parsing
- Interactive terminal-style loop
- Adding message history and multi-session memory
- Prompt templating with system + dynamic messages
- Token-based trimming for long chats
- Memory-enabled runnable chains

Each code cell below is followed by a brief markdown explanation describing its purpose.

> Security: All API keys have been removed. Replace placeholders like `Write your own password` with your own keys via environment variables.


In [98]:
!pip install -q langchain_google_genai

**Explanation:** Installs the LangChain Google Generative AI integration which provides the `ChatGoogleGenerativeAI` wrapper for Gemini models.

In [53]:
!pip install -q langchain

**Explanation:** Installs the core `langchain` package which provides abstractions for models, prompts, chains, and memory.

In [54]:
!pip install -q langchain_community

**Explanation:** Installs `langchain_community` which hosts integrations (loaders, tools, vector stores, etc.) maintained by the community.

In [55]:
import os

**Explanation:** Imports the `os` module so we can set environment variables for API keys in a secure and configurable way.

In [ ]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"  # Enables LangSmith tracing (optional)
os.environ["LANGCHAIN_API_KEY"] = "Write your own password"  # <-- Insert your LangSmith / LangChain API key
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "chatbot_with_langchain"
os.environ["GOOGLE_API_KEY"] = "Write your own password"  # <-- Insert your Google Generative AI (Gemini) API key

**Explanation:** Sets required environment variables. Replace the placeholder values (`Write your own password`) with your real API keys locally—never commit secrets.

In [57]:
import warnings
warnings.filterwarnings('ignore')

**Explanation:** Suppresses non-critical warnings to keep notebook output clean for readers.

In [58]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite-001",convert_system_message_to_human=True)

**Explanation:** Instantiates the Gemini chat model wrapper. `convert_system_message_to_human=True` adapts system prompts to the model's expected format.

In [59]:
# from google.colab import userdata
# GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
# os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

**Explanation:** Shows (commented) how you could fetch keys securely in Colab. Keys are intentionally not embedded for security.

In [60]:
model.invoke("hi").content

'Hi there! How can I help you today?'

**Explanation:** Simple one-off model invocation returning a greeting; demonstrates base `.invoke()` call returning a single message object whose `.content` holds text.

In [61]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parser.invoke(model.invoke("hi"))

'Hi there! How can I help you today?'

**Explanation:** Imports `StrOutputParser` to extract plain text from model responses and shows a simple parse of a greeting.

In [62]:
from langchain_core.messages import HumanMessage

**Explanation:** Imports `HumanMessage` (structured message object) to send user turns to the model in a standardized format.

In [63]:
while True:
  message = input("Write your query:")
  if message=="bye":
    print("Good Bye have a great day!")
    break
  else:
    print(parser.invoke(model.invoke([HumanMessage(content=message)])))

Write your query:what is capital of pakistan?
The capital of Pakistan is Islamabad.
Write your query:islamabad
Islamabad is the capital city of Pakistan. Here's a breakdown of key things about it:

**General Information:**

*   **Capital City:** It serves as the seat of the Pakistani government.
*   **Planned City:** Islamabad is a relatively new city, specifically designed and built to be the capital. This means it has a more organized layout compared to many older cities in Pakistan.
*   **Location:** Situated in the Potohar Plateau in the north-central part of Pakistan, near the Margalla Hills. It's close to the historical city of Rawalpindi.
*   **Population:** The population is estimated to be around 1.2 million, but it's constantly growing.
*   **Official Language:** Urdu and English.
*   **Climate:** Islamabad has a humid subtropical climate with four distinct seasons:
    *   **Summer:** Hot and humid, with temperatures often exceeding 40°C (104°F).
    *   **Monsoon:** The mon

**Explanation:** Simple blocking CLI loop: accepts user input, exits on `bye`, otherwise sends the message and prints the parsed model reply.

In [64]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

**Explanation:** Imports classes for maintaining and wrapping chat history (`InMemoryChatMessageHistory`, `RunnableWithMessageHistory`) plus `AIMessage` for simulating previous turns.

In [65]:
result = model.invoke(
    [
        HumanMessage(content="Hi! I'm cristinao"),
        AIMessage(content="Hello cristinao! How can I assist you today?"),
        HumanMessage(content="What's my name?")
    ]
)

**Explanation:** Demonstrates a stateless multi-turn style by manually passing prior human/AI messages; the model can't truly "remember" later unless we supply them again.

In [66]:
parser.invoke(result)

'Your name is cristinao. 😊'

**Explanation:** Parses the previous response into plain text using the output parser for readability.

In [67]:

store={}

**Explanation:** Initializes an in-memory dictionary to hold session-specific chat histories keyed by `session_id`.

In [68]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

**Explanation:** Helper that lazily creates and returns an `InMemoryChatMessageHistory` for a given `session_id`, enabling multi-session memory.

In [69]:
config = {"configurable": {"session_id": "firstchat"}}
model_with_memory=RunnableWithMessageHistory(model,get_session_history)

In [70]:
model_with_memory.invoke([HumanMessage(content="Hi! I'm Atif")],config=config).content

"Hi Atif! It's nice to meet you. How can I help you today?"

In [71]:
model_with_memory.invoke([HumanMessage(content="what is my name?")],config=config).content


'Your name is Atif. You told me that! 😊'

In [72]:
config = {"configurable": {"session_id": "secondtchat"}}

In [73]:
config

{'configurable': {'session_id': 'secondtchat'}}

In [74]:
model_with_memory.invoke([HumanMessage(content="what is my name?")],config=config).content

'I am a large language model, and I do not have a name. I am here to assist you with your questions.'

In [75]:
config = {"configurable": {"session_id": "firstchat"}}

In [76]:
config

{'configurable': {'session_id': 'firstchat'}}

In [77]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm Atif", additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Atif! It's nice to meet you. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-lite-001', 'safety_ratings': []}, id='run--4c5d7963-3e0c-4083-8f79-f47c5c039322-0', usage_metadata={'input_tokens': 7, 'output_tokens': 20, 'total_tokens': 27, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='what is my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Atif. You told me that! 😊', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-lite-001', 'safety_ratings': []}, id='run--e04b6478-9561-4ea7-be7d-43c60d7a5a5f-0', usage_metadata={'input_tokens': 31

In [78]:
model_with_memory.invoke([HumanMessage(content="what is my name?")],config=config).content

'Your name is Atif.'

In [79]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [80]:
prompt=ChatPromptTemplate.from_messages(
    [
      ("system","You are a helpful assistant. Answer all questions to the best of your ability.",),
      MessagesPlaceholder(variable_name="messages"),
    ]
)

In [81]:
chain = prompt | model

In [82]:
chain.invoke({"messages":[HumanMessage(content="Hi! I'm Atif")]})

AIMessage(content="Hi Atif! It's nice to meet you. How can I help you today? I'm ready for your questions.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-lite-001', 'safety_ratings': []}, id='run--62d6513e-b3af-4bf9-b159-76004b9c1723-0', usage_metadata={'input_tokens': 23, 'output_tokens': 28, 'total_tokens': 51, 'input_token_details': {'cache_read': 0}})

In [83]:
config = {"configurable": {"session_id": "thirdchat"}}

In [84]:
response=model_with_memory.invoke(
    [HumanMessage(content="Hi! I'm Atif"),
     ],config=config
)

In [85]:
response.content

"Hi Atif! It's nice to meet you. How can I help you today?"

In [86]:
response=model_with_memory.invoke(
    [HumanMessage(content="hi what is my name"),
     ],config=config
)
print(response.content)

Your name is Atif. I remember that you told me earlier! 😊


In [87]:

response=model_with_memory.invoke(
    [HumanMessage(content="what is 2+2?"),
     ],config=config
)
print(response.content)

2 + 2 = 4


In [88]:

response=model_with_memory.invoke(
    [HumanMessage(content="who is a pakistan cricket team caption?"),
     ],config=config
)
print(response.content)

The current captain of the Pakistan Men's Cricket team is **Babar Azam**.


In [89]:
response=model_with_memory.invoke(
    [HumanMessage(content="what is my name?"),
     ],config=config
)
print(response.content)

Your name is Atif! 


In [90]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm Atif", additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Atif! It's nice to meet you. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-lite-001', 'safety_ratings': []}, id='run--4c5d7963-3e0c-4083-8f79-f47c5c039322-0', usage_metadata={'input_tokens': 7, 'output_tokens': 20, 'total_tokens': 27, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='what is my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Atif. You told me that! 😊', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-lite-001', 'safety_ratings': []}, id='run--e04b6478-9561-4ea7-be7d-43c60d7a5a5f-0', usage_metadata={'input_tokens': 31

# trimming the conversation

In [91]:
from langchain_core.messages import SystemMessage, trim_messages

In [92]:
trimmer = trim_messages(
    max_tokens=60,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

In [93]:

messages = [
    HumanMessage(content="hi! I'm Atif"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [94]:
model.get_num_tokens_from_messages(messages)


52

In [95]:
trimmer.invoke(messages)

[HumanMessage(content="hi! I'm Atif", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [96]:
trimmed_message = trimmer.invoke(messages)


model.get_num_tokens_from_messages(trimmed_message)

52

In [97]:
prompt

ChatPromptTemplate(input_variables=['messages'], input_types={'messages': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.mes

In [99]:
trimmed_message

[HumanMessage(content="hi! I'm Atif", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [100]:
trimmed_message+[HumanMessage(content="what's my name?")]

[HumanMessage(content="hi! I'm Atif", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="what's my name?", additional_kwargs={}, response_metadata={})]

In [102]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

'Your name is Atif!'

In [103]:
esponse = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'Your name is Atif!'

In [104]:
model_with_memory = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)


In [106]:
config = {"configurable": {"session_id": "fourthchat"}}

In [107]:
response = model_with_memory.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content


'Your name is Atif! '

In [109]:
response = model_with_memory.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content


'You didn\'t explicitly ask me a math problem. You\'ve asked questions like "what\'s my name?" and "what math problem did I ask?". If you\'d like me to answer a math question, please feel free to ask!'

In [110]:
response = model_with_memory.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content


"I am designed to be a helpful assistant, but I don't have memory of past conversations. Therefore, I don't know what math problem you asked. \n\nTo help me answer, please tell me the math problem you'd like me to solve!"

# About the Author

<div style="background-color: #f8f9fa; border-left: 5px solid #28a745; padding: 20px; margin-bottom: 20px; border-radius: 5px;">
  <h2 style="color: #28a745; margin-top: 0; font-family: 'Poppins', sans-serif;">Muhammad Atif Latif</h2>
  <p style="font-size: 16px; color: #495057;">Data Scientist & Machine Learning Engineer</p>
  
  <p style="font-size: 15px; color: #6c757d; margin-top: 15px;">
    Passionate about building AI solutions that solve real-world problems. Specialized in machine learning,
    deep learning, and data analytics with experience implementing production-ready models.
  </p>
</div>

## Connect With Me

<div style="display: flex; flex-wrap: wrap; gap: 10px; margin-top: 15px;">
  <a href="https://github.com/m-Atif-Latif" target="_blank">
    <img src="https://img.shields.io/badge/GitHub-Follow-212121?style=for-the-badge&logo=github" alt="GitHub">
  </a>
  <a href="https://www.kaggle.com/matiflatif" target="_blank">
    <img src="https://img.shields.io/badge/Kaggle-Profile-20BEFF?style=for-the-badge&logo=kaggle" alt="Kaggle">
  </a>
  <a href="https://www.linkedin.com/in/muhammad-atif-latif-13a171318" target="_blank">
    <img src="https://img.shields.io/badge/LinkedIn-Connect-0077B5?style=for-the-badge&logo=linkedin" alt="LinkedIn">
  </a>
  <a href="https://x.com/mianatif5867" target="_blank">
    <img src="https://img.shields.io/badge/Twitter-Follow-1DA1F2?style=for-the-badge&logo=twitter" alt="Twitter">
  </a>
  <a href="https://www.instagram.com/its_atif_ai/" target="_blank">
    <img src="https://img.shields.io/badge/Instagram-Follow-E4405F?style=for-the-badge&logo=instagram" alt="Instagram">
  </a>
  <a href="mailto:muhammadatiflatif67@gmail.com">
    <img src="https://img.shields.io/badge/Email-Contact-D14836?style=for-the-badge&logo=gmail" alt="Email">
  </a>
</div>

---

# Project Summary & Overview

This notebook evolves a simple Gemini model call into a robust conversational system with:

1. Clean Environment Setup: Dependency installs and environment variable placeholders (no real keys committed).
2. Core LLM Invocation: Basic greeting + output parsing for clean text extraction.
3. Interactive Loop: A lightweight REPL to manually test responses.
4. Manual Multi-Turn Context: Passing prior messages explicitly (stateless approach demonstration).
5. Session Memory Abstraction: `RunnableWithMessageHistory` adds persistent state across turns.
6. Multi-Session Isolation: Distinct `session_id` values simulate multiple users without leakage.
7. Prompt Engineering: System + dynamic messages via `ChatPromptTemplate`.
8. Token Management: Trimming strategy keeps recent context inside a token budget for efficiency.
9. Chained Runnables: Composition of trimming -> prompt -> model -> memory for cleaner orchestration.
10. Referential Q&A: Model correctly recalls user-provided name and earlier math problem within a session.

Security & Best Practices:
- Secrets are NEVER hardcoded. Replace `Write your own password` locally with environment variables.
- In production, prefer a secret manager (Vault, AWS Secrets Manager, GCP Secret Manager) over inline assignment.
- Conversation trimming prevents context bloat and reduces latency/cost.

Why This Matters:
Building memory-aware assistants is essential for user experience. This template shows minimal moving parts while staying extensible (swap model, persist history, add tools, vector retrieval, etc.).

About the Creator:
Crafted by **Muhammad Atif Latif** — passionate about production-grade AI, scalable data pipelines, and applied ML that solves real user problems. Open to collaborations, speaking, and consulting opportunities.

---